In [1]:
#Basic Libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint
import sys
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
#Load Dataset
df=pd.read_csv('/home/smriti/Desktop/NLP/MITACS/Anxiety/Data/CSV/Reddit/rAnxiety13.csv')
#getting rid of NaN
df=df.replace(np.nan, '', regex=True)
#getting rid of deleted values
df['Text']=df['Text'].replace('[deleted]','')
#Combining title and text
df["Post"] = df["Title"] + df["Text"]
#Now that we don't need Title or Text, we drop those columns before saving the file
df=df.drop(['Title', 'Text'], axis = 1)

/home/smriti/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [4]:
df=df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
df.head()

,Number of Comments,Date Posted,Post
0,38,2013-05-03 16:31:35 EDT-0400,I think this adequately describes what I go th...
1,17,2013-05-02 19:41:23 EDT-0400,My sister told me she saw this and thought of ...
2,32,2013-05-01 19:18:32 EDT-0400,Just spent an hour in a coffee shop on my own ...
3,60,2013-05-04 15:53:25 EDT-0400,Caution to anyone planning to watch Iron Man 3
4,31,2013-05-02 17:53:07 EDT-0400,DAE have this happen to them?


In [6]:
# Convert to list
data=df.Post.values.tolist()
# Remove new line characters
data=[re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data=[re.sub("\'", "", sent) for sent in data]

In [7]:
#Function to clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

In [8]:
#Clean up text
data_words=list(sent_to_words(data))
print(data_words[:1])

[['think', 'this', 'adequately', 'describes', 'what', 'go', 'through', 'every', 'single', 'day']]


In [9]:
# Build the bigram and trigram models
bigram=gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram=gensim.models.Phrases(bigram[data_words], threshold=100)

In [10]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod=gensim.models.phrases.Phraser(trigram)

In [11]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['think', 'this', 'adequately', 'describes', 'what', 'go', 'through', 'every_single', 'day']


In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [13]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [22]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

# Create Dictionary needed for topic modelling
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=62,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

[['think', 'adequately', 'describe', 'go', 'day']]


In [23]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(61,
  '0.000*"genre" + 0.000*"flare" + 0.000*"upside" + 0.000*"thriller" + '
  '0.000*"summary" + 0.000*"stigmata" + 0.000*"select" + 0.000*"missing" + '
  '0.000*"hardest" + 0.000*"hormone"'),
 (17,
  '0.000*"genre" + 0.000*"flare" + 0.000*"upside" + 0.000*"thriller" + '
  '0.000*"summary" + 0.000*"stigmata" + 0.000*"select" + 0.000*"missing" + '
  '0.000*"hardest" + 0.000*"hormone"'),
 (46,
  '0.000*"genre" + 0.000*"flare" + 0.000*"upside" + 0.000*"thriller" + '
  '0.000*"summary" + 0.000*"stigmata" + 0.000*"select" + 0.000*"missing" + '
  '0.000*"hardest" + 0.000*"hormone"'),
 (48,
  '0.000*"genre" + 0.000*"flare" + 0.000*"upside" + 0.000*"thriller" + '
  '0.000*"summary" + 0.000*"stigmata" + 0.000*"select" + 0.000*"missing" + '
  '0.000*"hardest" + 0.000*"hormone"'),
 (58,
  '0.000*"genre" + 0.000*"flare" + 0.000*"upside" + 0.000*"thriller" + '
  '0.000*"summary" + 0.000*"stigmata" + 0.000*"select" + 0.000*"missing" + '
  '0.000*"hardest" + 0.000*"hormone"'),
 (44,
  '0.000*"genr

In [24]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -20.06869005650696


In [25]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3592376790249025


In [32]:
num_topics = 62
topic_words = []
for i in range(num_topics):
    tt = lda_model.get_topic_terms(i,10)
    topic_words.append([id2word[pair[0]] for pair in tt])

In [33]:
for i in range(0,62):
    print(topic_words[i])

['genre', 'flare', 'upside', 'thriller', 'summary', 'stigmata', 'select', 'missing', 'hardest', 'hormone']
['ahead', 'hardest', 'butt', 'upside', 'thriller', 'summary', 'stigmata', 'select', 'missing', 'hormone']
['genre', 'flare', 'upside', 'thriller', 'summary', 'stigmata', 'select', 'missing', 'hardest', 'hormone']
['kind', 'big', 'hit', 'tired', 'wake', 'awesome', 'tonight', 'screw', 'working', 'floor']
['testing', 'genre', 'flare', 'upside', 'thriller', 'summary', 'stigmata', 'select', 'missing', 'hardest']
['little', 'do', 'feeling', 'level', 'state', 'set', 'entire', 'negative', 'fast', 'suggestion']
['laugh', 'shirt', 'stigmata', 'summary', 'select', 'hardest', 'dominance', 'upside', 'butt', 'thriller']
['people', 'help', 'make', 'life', 'would', 'want', 'find', 'try', 'thing', 'really']
['example', 'quite', 'bus', 'asshole', 'bit', 'least', 'less', 'instead', 'situation', 'often']
['whenever', 'word', 'negative', 'quickly', 'man', 'period', 'cognitive', 'rid', 'improvement', '

In [35]:
df=pd.DataFrame({'Year':[],'Source':[],'Topic_ID':[],'Most_freq_words':[]})
df.head()

,Most_freq_words,Source,Topic_ID,Year


In [36]:
df['Most_freq_words']=topic_words

In [37]:
df = df.assign(Year='2013')
df = df.assign(Source='Reddit')

In [39]:
ls=[]
for i in range(0,62):
    ls.append(i)
df['Topic_ID']=ls

In [40]:
df.head()

,Most_freq_words,Source,Topic_ID,Year
0,"[genre, flare, upside, thriller, summary, stig...",Reddit,0,2013
1,"[ahead, hardest, butt, upside, thriller, summa...",Reddit,1,2013
2,"[genre, flare, upside, thriller, summary, stig...",Reddit,2,2013
3,"[kind, big, hit, tired, wake, awesome, tonight...",Reddit,3,2013
4,"[testing, genre, flare, upside, thriller, summ...",Reddit,4,2013


In [41]:
df.to_csv("topic_words_r2013.csv")